In [1]:
from docx import Document as DocxDocument
from langchain.docstore.document import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
import pickle, os
from datetime import datetime
from langchain_openai import ChatOpenAI
from langchain.chains.retrieval_qa.base import RetrievalQA
from dotenv import load_dotenv
import requests
from bs4 import BeautifulSoup
from langchain.docstore.document import Document
from datetime import datetime
import re
from pathlib import Path
from __future__ import annotations
from datetime import datetime
from pathlib import Path
from typing import List

import re, requests, html
from bs4 import BeautifulSoup
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from urllib.parse import urljoin, urlparse
load_dotenv()

True

In [2]:
docs = []
chunks = []
CHUNK_TOKENS   = 200
CHUNK_OVERLAP  = 20
SEPARATORS     = ["\n\n", "\n", ". ", " "]

splitter = RecursiveCharacterTextSplitter(
    chunk_size      = CHUNK_TOKENS * 6,   # ~400 tokens ≈ 2400 chars
    chunk_overlap   = CHUNK_OVERLAP * 6,  # keep ~40‑token context
    separators      = SEPARATORS,
)

In [3]:
def _clean(text: str) -> str:
    """Remove common TOC dots, extra whitespace, etc."""
    text = re.sub(r'\.{5,}\s*\d+', '', text)   # dotted TOC lines
    text = re.sub(r'\n{3,}', '\n\n', text)     # huge blank gaps
    return text.strip()

def load_and_chunk_docx(path: str | Path) -> list[Document]:
    docx = DocxDocument(path)
    file_name = Path(path).name
    uploaded  = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # 1️⃣ Collect paragraphs with their nearest heading
    current_heading = "Untitled"
    paragraphs      = []
    for para in docx.paragraphs:
        style = para.style.name
        text  = para.text.strip()
        if not text:
            continue

        if style.startswith("Heading"):          # e.g., 'Heading 1'
            current_heading = text
            continue

        paragraphs.append({"heading": current_heading, "text": text})

    # 2️⃣ Group paragraphs by heading to reduce splitter fragmentation
    grouped = {}
    for p in paragraphs:
        grouped.setdefault(p["heading"], []).append(p["text"])

    docs = []
    for heading, para_list in grouped.items():
        combined = _clean("\n".join(para_list))
        if not combined:
            continue

        # 3️⃣ Split into semantic chunks
        chunks = splitter.split_text(combined)
        for idx, chunk in enumerate(chunks):
            meta = {
                "source": file_name,
                "section": heading,
                "chunk_index": idx,
                "uploaded_at": uploaded,
            }
            docs.append(Document(page_content=chunk, metadata=meta))

    return docs

In [5]:
def save_faiss_index():
    if os.path.exists("faiss.idx"):
        print("FAISS index already exists, skipping creation.")
        return

    print("Creating FAISS index…")
    chunks = []

    # local .docx files
    training = "training"
    for fname in os.listdir(training):
        if fname.startswith("."):          # skip hidden files
            continue
        docs = load_and_chunk_docx(os.path.join(training, fname))
        chunks.extend(splitter.split_documents(docs))

    # # 2. live URLs
    # for url in urls:
    #     try:
    #         docs = load_and_chunk_docx(url)           # << only the URL string here
    #         chunks.extend(splitter.split_documents(docs))
    #     except Exception as e:
    #         print(f"[WARN] {url} skipped – {e}")

    print(f"{len(chunks)} chunks total")
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    db = FAISS.from_documents(chunks, embeddings)
    db.save_local("faiss.idx")
    print("FAISS index created and saved.")

In [7]:
save_faiss_index()

Creating FAISS index…
79 chunks total


d:\Ai\ai-env\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


FAISS index created and saved.
